In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from fbprophet import Prophet
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("../input/web-traffic-time-series-forecasting/train_1.csv.zip")
keys = pd.read_csv("../input/web-traffic-time-series-forecasting/key_1.csv.zip")
ss = pd.read_csv("../input/web-traffic-time-series-forecasting/sample_submission_1.csv.zip")

In [ ]:
train.head()

In [ ]:
# Drop Page column
X_train = train.drop(['Page'], axis=1)
X_train.head()

In [ ]:
# Check the data
X_train.isnull().any().describe()

Prophet ignores nan value for calculation, so nan values does not affect predictions. For I replace null values into 0 because it may affect the calculation.

In [ ]:
y = X_train.to_numpy()[0]
df = pd.DataFrame({ 'ds': X_train.T.index.values, 'y': y})

In [ ]:
df.tail()

In [ ]:
df['ds']=pd.to_datetime(df['ds'],format='%Y-%m')

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(df.ds,df.y)

In [ ]:
# With outliers
m = Prophet()
m.fit(df)
future = m.make_future_dataframe(periods=10)
forecast = m.predict(future)
m.plot(forecast);

In [ ]:
# Remove outliers
y = X_train.dropna(0).to_numpy()[0] # Replace NaN to 0 for list comprehension
y = [ None if i >= np.percentile(y, 95) or i <= np.percentile(y, 5) else i for i in y ]
df_na = pd.DataFrame({ 'ds': X_train.T.index.values, 'y': y})

In [ ]:
# With outliers
m = Prophet()
m.fit(df_na)
future = m.make_future_dataframe(periods=10)
forecast = m.predict(future)
m.plot(forecast);

In [ ]:
future.tail()

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()